# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [337]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.pipeline import make_pipeline

### 데이터 셋 읽어오기

In [338]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [339]:
df_train.head(10) # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,...,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,...,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,...,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [340]:
df_test.head() # 테스트 데이터 살펴보기

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False


## 2. 데이터 전처리

### 레이블 인코딩

In [342]:
# df_train = df_train.drop_duplicates()
# Assuming df_train is your DataFrame
# Replace missing values in 'customer_country' with the mode of each 'lead_owner' group
mode_fill = df_train.groupby('lead_owner')['customer_country'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['customer_country'] = df_train['customer_country'].fillna(mode_fill)

mode_fill = df_train.groupby('lead_owner')['customer_country.1'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['customer_country.1'] = df_train['customer_country.1'].fillna(mode_fill)

mode_fill = df_train.groupby('customer_position')['inquiry_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['inquiry_type'] = df_train['inquiry_type'].fillna(mode_fill)

mode_fill = df_train.groupby('customer_position')['inquiry_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['inquiry_type'] = df_train['inquiry_type'].fillna(mode_fill)

mode_fill = df_train.groupby('customer_type')['inquiry_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df_train['inquiry_type'] = df_train['inquiry_type'].fillna(mode_fill)

# This code will fill the missing values in 'customer_country' with the mode of the corresponding 'lead_owner' group.
# If there is no mode (i.e., all values are missing) for a specific 'lead_owner', it will remain missing.

# Example:
# Before:
# df_train:
#    lead_owner  customer_country
# 0      owner1               USA
# 1      owner2               NaN
# 2      owner1               NaN
# 3      owner2               NaN

# After:
# df_train:
#    lead_owner  customer_country
# 0      owner1               USA
# 1      owner2               USA
# 2      owner1               USA
# 3      owner2               NaN  # If all values for owner2 are missing, it remains missing

# Count the number of missing values in 'customer_country'
missing_values_count = df_train['customer_country'].isnull().sum()

print("Number of missing values in customer_country:", missing_values_count)

missing_values_count = df_train['customer_country.1'].isnull().sum()

print("Number of missing values in customer_country.1:", missing_values_count)

missing_values_count = df_train['inquiry_type'].isnull().sum()

print("Number of missing values in inquiry_type: ", missing_values_count)

# df_train.to_csv('train_customer_country_filled.csv', index=False)


Number of missing values in customer_country: 5
Number of missing values in customer_country.1: 5
Number of missing values in inquiry_type:  8


In [343]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [344]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

print(df_train.isna().sum()/len(df_train))

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

bant_submit                0.000000
customer_country           0.000095
business_unit              0.000000
com_reg_ver_win_rate       0.758609
customer_idx               0.000000
customer_type              0.741401
enterprise                 0.000000
historical_existing_cnt    0.766390
id_strategic_ver           0.941086
it_strategic_ver           0.979763
idit_strategic_ver         0.920849
customer_job               0.334911
lead_desc_length           0.000000
inquiry_type               0.000151
product_category           0.324934
product_subcategory        0.838688
product_modelname          0.836511
customer_country.1         0.000095
customer_position          0.000000
response_corporate         0.000000
expected_timeline          0.500615
ver_cus                    0.000000
ver_pro                    0.000000
ver_win_rate_x             0.690230
ver_win_ratio_per_bu       0.746739
business_area              0.690230
business_subarea           0.907483
lead_owner                 0

다시 학습 데이터와 제출 데이터를 분리합니다.

In [345]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]



### 피쳐의 중요도 확인하기

### 상관계수 : 필요없는 특성 제거

In [346]:
# drop_column = list(df_train.columns)
# print(drop_column)

# # 상관계수 계산
# corr_matrix = df_train.corr()
# corr_matrix["is_converted"].sort_values(ascending=False)
    

['bant_submit', 'customer_country', 'business_unit', 'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise', 'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job', 'lead_desc_length', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_country.1', 'customer_position', 'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area', 'business_subarea', 'lead_owner', 'is_converted']


is_converted               1.000000
com_reg_ver_win_rate       0.350275
enterprise                 0.123788
lead_desc_length           0.108638
ver_win_ratio_per_bu       0.107243
lead_owner                 0.094012
ver_cus                    0.063905
business_unit              0.052918
customer_country           0.041037
customer_country.1         0.041037
response_corporate         0.034534
product_category           0.033170
expected_timeline          0.009471
historical_existing_cnt    0.009264
ver_pro                    0.005278
customer_job              -0.013146
bant_submit               -0.015725
customer_position         -0.019507
business_area             -0.023491
inquiry_type              -0.030124
ver_win_rate_x            -0.056763
product_modelname         -0.058324
customer_idx              -0.058334
product_subcategory       -0.070236
business_subarea          -0.105395
customer_type             -0.106342
id_strategic_ver                NaN
it_strategic_ver            

### is_converted 와 가장 관련 없는 칼럼을 지운다.


In [347]:
df_train = df_train.drop(columns=['expected_timeline', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver'])
df_test = df_test.drop(columns=['expected_timeline', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver'])
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,8186,0,0.066667,32160,9,0,NaN,401,62,...,46,33,1,0,0.003079,0.026846,0,28,0,True
1,1.0,7612,0,0.066667,23122,9,0,12.0,292,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.0,5959,0,0.088889,1755,9,0,144.0,153,56,...,77,21,1,0,0.003079,0.026846,0,17,2,True
3,1.0,3219,0,0.088889,4919,9,0,NaN,159,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.0,5337,0,0.088889,17126,28,0,NaN,81,97,...,77,21,0,0,0.003079,0.026846,0,86,4,True


### 2-2. 학습, 검증 데이터 분리

In [348]:
df_train.fillna(df_train.mean())

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,8186,0,0.066667,32160,9,0,17.739546,401,62,...,46,33,1,0,0.003079,0.026846,0,28,0,True
1,1.00,7612,0,0.066667,23122,9,0,12.000000,292,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.00,5959,0,0.088889,1755,9,0,144.000000,153,56,...,77,21,1,0,0.003079,0.026846,0,17,2,True
3,1.00,3219,0,0.088889,4919,9,0,17.739546,159,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.00,5337,0,0.088889,17126,28,0,17.739546,81,97,...,77,21,0,0,0.003079,0.026846,0,86,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59291,0.75,4668,0,0.040000,19266,28,0,17.739546,153,50,...,61,32,0,0,0.000026,0.028777,8,78,42,False
59292,1.00,3064,0,0.097763,36256,31,0,17.739546,213,171,...,36,22,0,0,0.000026,0.028777,8,86,774,False
59296,0.75,7943,0,0.040000,19249,28,0,17.739546,153,34,...,46,35,0,0,0.000026,0.028777,8,86,125,False
59297,1.00,11376,0,0.040000,40327,31,0,17.739546,159,377,...,76,35,0,0,0.000026,0.028777,8,86,134,False


In [349]:
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,8186,0,0.066667,32160,9,0,NaN,401,62,...,46,33,1,0,0.003079,0.026846,0,28,0,True
1,1.0,7612,0,0.066667,23122,9,0,12.0,292,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.0,5959,0,0.088889,1755,9,0,144.0,153,56,...,77,21,1,0,0.003079,0.026846,0,17,2,True
3,1.0,3219,0,0.088889,4919,9,0,NaN,159,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.0,5337,0,0.088889,17126,28,0,NaN,81,97,...,77,21,0,0,0.003079,0.026846,0,86,4,True


In [350]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 정의 

In [351]:
model = DecisionTreeClassifier()
# model = RandomForestClassifier()

In [352]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))


In [353]:
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

In [354]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[ 748  186]
 [ 193 9438]]

정확도: 0.9641
정밀도: 0.7949
재현율: 0.8009
F1: 0.7979


### GridSearchCV를 이용한 하이퍼파라미터 튜닝

In [355]:
# 탐색할 하이퍼파라미터 설정
param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(x_train.fillna(0), y_train)

# 최적의 하이퍼파라미터 출력
print('최고 평균 정확도 수치 : {0:.4f}'.format(grid_search.best_score_))
print("Best Hyperparameters:", grid_search.best_params_)

최고 평균 정확도 수치 : 0.7668
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [356]:
best_tree = grid_search.best_estimator_

### 모델 학습

In [357]:
# model.fit(x_train.fillna(0), y_train)
#
# rf_clf1 = RandomForestClassifier(n_estimators = 100,
#                                  max_depth = 12,
#                                  random_state = 0,)
# rf_clf1.fit(x_train.fillna(0), y_train)
# pred = rf_clf1.predict(x_val.fillna(0))
# print(classification_report(y_val, pred))

### 모델 성능 보기

In [358]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [359]:
# pred = model.predict(x_val.fillna(0))
# pred = rf_clf1.predict(x_val.fillna(0))
pred = best_tree.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[ 761  173]
 [ 195 9436]]

정확도: 0.9652
정밀도: 0.7960
재현율: 0.8148
F1: 0.8053


In [360]:
# model.fit(x_train.fillna(0),y_train)

## 4. 제출하기

In [361]:
# # 최적의 모델로 예측
# best_dt_model = grid_search.best_estimator_
# y_pred = best_dt_model.predict(x_val.fillna(0))
#
# # 성능 평가
# print("\nClassification Report:")
# print(classification_report(y_val, y_pred))

### 테스트 데이터 예측

In [362]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [363]:
# test_pred = model.predict(x_test.fillna(0))
# test_pred = rf_clf1.predict(x_test.fillna(0))
test_pred = best_tree.predict(x_test.fillna(0))

sum(test_pred) # True로 예측된 개수

967

### 제출 파일 작성

In [364]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**